In [1]:
from itertools import chain
from random import random

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from society.action import Action
from society.agent import Agent, TrainableAgent
from society.simulations.weighted import WeightedNetworkSimulation
from society.strategies.gameplay.constant import AllC, AllD
from society.strategies.gameplay.qlearning import TabularQLearningGameplayStrategy
from society.strategies.gameplay.random import RandomGameplayStrategy
from society.strategies.gameplay.tft import TitForTat
from society.visualisation.network import *

plt.rcParams["figure.figsize"] = (10, 6)

In [2]:
def generate_population(communities, size):
    population = communities * size

    agents = [
        Agent(TabularQLearningGameplayStrategy(lookback=2, epsilon=0.2), i, population)
        for i in range(population)
    ]

    G = nx.connected_caveman_graph(communities, size)

    weights_matrix = np.zeros((population, population))
    for u, v, d in G.edges(data=True):
        weights_matrix[u, v] = weights_matrix[v, u] = 1.0
        try:
            d["weight"] = weights_matrix[u, v]
        except:
            d["weight"] = 0

    return agents, weights_matrix, G

In [3]:
def compute_policies(agents):
    policies = [
        agent.gameplay_strategy._q_table.argmax(axis=-1)
        for agent in agents
    ]

    return [
        "".join(
            [
                ("C", "D")[policy[i, j]]  # , k
                for i in range(4)
                for j in range(4)
                # for k in range(4)
            ]
        )
        for policy in policies
    ]

def calculate_cooperativeness(history):
    count = history.count(Action.COOPERATE)

    return count / len(history)

In [4]:
# PAYOFF_LABELS = ["(C, C)", "(C, D)", "(D, C)", "(D, D)"]

PAYOFF_LABELS = ["R", "S", "T", "P"]

POPULATION = (2, 32)
ROUNDS = 25_000

run_rewards = []
run_cumulative_rewards = []
run_action_histories = []
run_reward_histories = []
run_policies = []

for run in range(20):
    # Generate a new population
    agents, weights_matrix, G = generate_population(*POPULATION)

    # Run a number of rounds
    sim = WeightedNetworkSimulation(agents, weights_matrix)

    # Run the simulation
    sim.reset()
    for i in tqdm(range(ROUNDS), desc=f"Run {run + 1}"):
        sim.play_round(train=True)

    # Store results
    run_rewards.append(sim.rewards)
    run_action_histories.append(sim.action_histories)
    run_reward_histories.append(sim.reward_histories)
    run_policies.append(compute_policies(agents))

    # Output statistics for the best and worst agents
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(sim.reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    tqdm.write(
        f"BEST AGENT: {cumulative_rewards[0][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[0][0]])))}, {run_policies[-1][cumulative_rewards[0][0]]})"
    )
    tqdm.write(
        f"WORST AGENT: {cumulative_rewards[-1][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[-1][0]])))}, {run_policies[-1][cumulative_rewards[-1][0]]})"
    )


Run 1:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 130175 (0.416669988440228, DDDDDCCCCDCDCDCC)
WORST AGENT: 111009 (0.624059082516688, DDCCCDCDCDCCDDCC)


Run 2:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 135714 (0.3321997279347043, DDCCCCCCCDCCCDCC)
WORST AGENT: 109418 (0.6322574198061357, DDCCDDCCCDCCCDCC)


Run 3:   0%|          | 0/25000 [00:00<?, ?it/s]

...

BEST AGENT: 140842 (0.32628133482356697, DCDDDDCDDDCDCDDD)
WORST AGENT: 108201 (0.6100555102365651, DDCDCDCDCDCCDDCC)


Run 19:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 135348 (0.3555671217520118, DDCDDCCDCDDCDDDD)
WORST AGENT: 109040 (0.6785005910283894, DDCCCDCCCDCCCDDC)


Run 20:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 135664 (0.3692737430167598, DDDDDDCDCDCDDCCD)
WORST AGENT: 107987 (0.6905878993105137, CDCCCDCCCDCCCDCC)


In [14]:
from datetime import datetime
import pickle

with open(f"results - {datetime.isoformat(datetime.now()).replace(':', '-')}.pickle", "wb") as f:
    pickle.dump({
        "rewards": run_rewards,
        "cumulative_rewards": run_cumulative_rewards,
        # "action_histories": run_action_histories,
        "reward_histories": run_reward_histories,
        "policies": run_policies,
    }, f)


In [5]:
cumulative_reward_policy_ranks = {}
cumulative_reward_policies = {}

for policies, reward_histories in zip(run_policies, run_reward_histories):
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    for rank, (i, cumulative_reward) in enumerate(cumulative_rewards):
        policy = policies[i]

        if policy not in cumulative_reward_policy_ranks:
            cumulative_reward_policy_ranks[policy] = []
            cumulative_reward_policies[policy] = []

        cumulative_reward_policy_ranks[policy].append(rank + 1)
        cumulative_reward_policies[policy].append(cumulative_reward)

mean_cumulative_reward_policy_ranks = {policy: np.mean(ranks) for policy, ranks in cumulative_reward_policy_ranks.items()}
mean_cumulative_reward_policies = {policy: np.mean(rewards) for policy, rewards in cumulative_reward_policies.items()}

In [6]:
for policy in sorted(mean_cumulative_reward_policy_ranks, key=lambda x: mean_cumulative_reward_policy_ranks[x]):
    print(f"{mean_cumulative_reward_policy_ranks[policy]:<24} {policy}")

1.0                      DDDDDCCCCDCDCDCC
1.0                      DDCCCCCCCDCCCDCC
1.0                      DDDCDDDDCDDCDDDD
1.0                      CDCDCDDDCCDDCCDD
1.0                      CDCCDCCDDCDCCDCC
1.0                      CDDCDDDDCDCDDDDC
1.0                      DDCDCCDDCDDCDDDD
1.0                      CDCCCCCDDDDCDDCD
1.0                      CDCDDCDCCDCCDDCD
1.0                      DDCDCCCDDDCDCDCD
1.0                      CDDDDDCDCCDDDDCC
1.0                      DDDCCCDDDDDDDCDD
1.0                      CDCCDDCCCDCCCDDC
1.0                      DDCCCCCCCDCCDDCC
1.0                      DCDDDDCDDDCDCDDD
1.0                      DDCDDCCDCDDCDDDD
1.0                      DDDDDDCDCDCDDCCD
2.0                      DDCCCDCDDDCCDDCD
2.0                      DDCCCDDCCDCCDCCD
2.0                      CCCCCDCDDCDDDCDD
2.0                      DDDCCCDCDDCDDDCD
2.0                      DCDCCDDCDCCDDCCD
2.0                      DCCCCDCDDDDCDCCD
2.0                      DDCDCDCDD

In [7]:
for policy in sorted(mean_cumulative_reward_policies, key=lambda x: mean_cumulative_reward_policies[x], reverse=True):
    print(f"{mean_cumulative_reward_policies[policy]:<24} {policy}")

143042.0                 CDDDDDCDCCDDDDCC
140842.0                 DCDDDDCDDDCDCDDD
140559.0                 DDDCDDDDCDDCDDDD
140525.0                 DDCDDCCCCDCCCCCD
139991.0                 DDCDCCDDCDDCDDDD
138447.0                 CDCDCDDDCCDDCCDD
137415.0                 DCCCCDCDDDDCDCCD
136531.0                 DDDCCCDCDDCDDDCD
136366.0                 CDDCDDDDCDCDDDDC
136321.0                 DDDCCCDDDDDDDCDD
135714.0                 DDCCCCCCCDCCCDCC
135664.0                 DDDDDDCDCDCDDCCD
135531.0                 DDCDCCCDDDCDCDCD
135530.0                 CCCCCDCDDCDDDCDD
135348.0                 DDCDDCCDCDDCDDDD
135203.0                 DCCCDCCCDDCDDDCC
135033.0                 CDCDDCCCDDCDCDCD
134738.0                 DDCDCCCDDDDCDDCD
134428.0                 DDCDCCCCCDCCCDCD
134389.0                 CDCDDCDCCDCCDDCD
134272.0                 DDCCDDCCDDCCDDDD
134264.0                 DDDCCDCCCDCDDDCD
134234.0                 DDDDDDCDDDDCDDCC
133590.0                 CDCDCDCDD